# 신경망 학습 (손실함수)

기계 학습 : 이미지 --> 사람이 생각한 알고리즘, 사람이 생각한 특징  --> 기계학습 --> 결과
    
딥러닝 : 이미지 --> 신경망(딥러닝) --> 결과
    
기계학습에서는 알고리즘을 짜거나 피쳐(특징)을 뽑을 때 사람이 개입해야하지만 딥러닝에서는 그런 과정조차 기계가 한다

딥러닝에서는 가지고 있는 데이터를 training_set 과 test_set으로 나눈다 : 범용성을 갖추기 위해서

--> 만약 training_set으로만 데이터를 모두 사용하여 학습 할 경우 overfitting이 일어날 가능성이 매우 높다

### 손실함수

최적의 매개변수 값을 탐색하는 지표(과정)

일반적으로 오차제곱합(SSE)과 교차 엔트로피 오차(Cross_Entropy_Error)를 사용

#### 오차제곱합(SSE)

In [4]:
def sum_squares_error(y,t):
    return 0.5 * np.sum((y-t)**2)

#### 교차 엔트로피 오차(CEE)

In [5]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta))  #매우 작은 값 delta를 추가한 이유 : t가 0이된다면 np.log(t)는 정의되지 않으므로 계산이 중단된다. 그런 상황을 방지하기 위함이다.

#### 미니배치 학습 : 전체 데이터의 일부만 무작위로 뽑아서 학습. 데이터의 크기가 너무 클 때 사용 

In [8]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)


#이 train_data에서 무작위로 10장 추출하기
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) #train_size인 60000에서 batch_size인 10개의 수를 무작위로 뽑아내는 것
x_batch = x_train[batch_mask]  #뽑은 10개의 수를 인덱스로 사용하여 x_batch 추출
t_batch = t_train[batch_mask]

#### 교차 엔트로피 오차 구현 (batch_size 이용가능 ver.)

In [10]:
def cross_entropy_error(y,t):
    if y.ndim == 1:  #y가 1차원이라면, 즉 데이터 하나당 교차 엔트로피를 구하는 것이라면
        t = t.reshape(1,t.size)  #그에 따라 t,y데이터 reshape
        y = y.reshape(1,y.size)
    
    batch_size = y.shape[0]
    
    return -np.sum(t*np.log(y+1e-7)) / batch_size

#### y_label이 one_hot이 아닌 경우 (숫자로 주어질 경우)

In [17]:
def cross_entropy_error(y,t):
    if y.ndim == 1:  #y가 1차원이라면, 즉 데이터 하나당 교차 엔트로피를 구하는 것이라면
        t = t.reshape(1,t.size)  #그에 따라 t,y데이터 reshape
        y = y.reshape(1,y.size)
    batch_size = y.shape[0]
    
    return -np.sum(np.log(y[np.arange(batch_size),t] + 1e-7)) / batch_size  #np.arnage(batch_size)는 0~ batch_size -1의 배열을 생성한다. 각 데이터의 정답 레이블에 해당하는 신경망의 출력 추출

정확도가 아닌 손실함수를 매개변수의 초기화 지표로 사용하는 이유 : 정확도를 지표로 한다면 매개변수의 미분이 대부분의 장소에서 0이 된다.

--> 정확도 지표 사용시에는 변화가 불연속적이며, 미소한 변화에는 반응을 하지않는다. 미분을 이용한 초기화이기 때문에 잘 맞지 않음.

--> 손실함수는 연속적이고 기울기가 0이 되지 않기에 사용이 용이하다.